In [120]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import fibsem
from fibsem.imaging import masks
from fibsem.imaging import utils   
from autoscript_sdb_microscope_client.structures import AdornedImage
from PIL import Image
import logging
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage import data
from skimage import filters
from skimage.color import rgb2gray
import imageio
import cv2
import glob
import itertools
import pandas as pd
from pprint import pprint
import plotly.express as px
import scipy
import statsmodels
import ipywidgets
from dataclasses import dataclass


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
def xcorr(
    img1: np.ndarray, 
    img2: np.ndarray, 
    use_mask: bool = True, 
    use_threshold: bool = True) -> np.ndarray:

    if img1.data.shape != img2.data.shape:
        err = f"Image 1 {img1.data.shape} and Image 2 {img2.data.shape} need to have the same shape"
        logging.error(err)
        raise ValueError(err)

    if use_mask:
        # Create Fourier Transform WITH Bandpass Filter
        pixelsize_img1 = img1.data.shape
        pixelsize_img2 = img2.data.shape
        img1_mask = masks.create_bandpass_mask(pixelsize_img1, 100, 4, 3)
        img2_mask = masks.create_bandpass_mask(pixelsize_img2, 100, 4, 3)
        img1fft = np.fft.ifftshift(img1_mask * np.fft.fftshift(np.fft.fft2(img1.data)))
        img2fft = np.fft.ifftshift(img2_mask * np.fft.fftshift(np.fft.fft2(img2.data)))
        assert img1_mask is not None, "Mask1 can not be None when using a Mask"
        assert img2_mask is not None, "Mask2 can not be None when using a Mask"

    else:
        # Create Fourier Transform
        img1fft = np.fft.ifftshift(np.fft.fftshift(np.fft.fft2(img1.data)))
        img2fft = np.fft.ifftshift(np.fft.fftshift(np.fft.fft2(img2.data)))

    # Do some shady normalization(?)
    n_pixels1 = img1.data.shape[0] * img1.data.shape[1]
    img1fft[0, 0] = 0
    tmp = img1fft * np. conj(img1fft)
    img1fft = n_pixels1 * img1fft / np.sqrt(tmp.sum())
    

    n_pixels2 = img2.data.shape[0] * img2.data.shape[1]
    img2fft[0, 0] = 0
    tmp = img2fft * np. conj(img2fft)
    img2fft = n_pixels2 * img2fft / np.sqrt(tmp.sum())


    if use_threshold:
        #Create threshold (Otsu's method) in abs Space
        abs1 = np.fft.fftshift(np.fft.ifftshift(np.fft.ifft2(img1fft)))
        abs2 = np.fft.fftshift(np.fft.ifftshift(np.fft.ifft2(img2fft)))
        otsu1 = filters.threshold_otsu(np.abs(abs1))
        otsu2 = filters.threshold_otsu(np.abs(abs2))

         #Create segmented binary Img
        binary1 = (abs1 > otsu1)*1
        binary2 = (abs2 > otsu2)*1
        
        #Back to Fourier Space for cross-correlation
        fft1 = np.fft.ifftshift(np.fft.fftshift(np.fft.fft2(binary1)))
        fft2 = np.fft.ifftshift(np.fft.fftshift(np.fft.fft2(binary2)))

        # Cross-correlate the two images
        corr = np.real(np.fft.fftshift(np.fft.ifft2(fft1 * np.conj(fft2))))

        #Cross-correlation center and shift from center
        maxX, maxY = np.unravel_index(np.argmax(corr), corr.shape)
        cen = np.asarray(corr.shape) / 2
        err = np.array(cen - [maxX, maxY], int)
        valMax = np.amax(corr)
        return corr, valMax, cen, err,

    else:
        # Cross-correlate the two images
        corr = np.real(np.fft.fftshift(np.fft.ifft2(img1fft * np.conj(img2fft))))
       
        #Cross-correlation center and shift from center
        maxX, maxY = np.unravel_index(np.argmax(corr), corr.shape)
        cen = np.asarray(corr.shape) / 2
        err = np.array(cen - [maxX, maxY], int)
        valMax = np.amax(corr)
        return corr, valMax, cen, err

In [179]:
neg = []
t = glob.glob("../xcorrtest/negative test/*.tif")
for j in t:
    read = mpimg.imread(j)
    read = AdornedImage(read, None)
    neg.append(read)
negate = np.array(neg)

data1 = []
files = glob.glob("../xcorrtest/*.tif")
for file in files:
    img = mpimg.imread(file)
    img = AdornedImage(img, None)
    data1.append(img)
arr = np.array(data1)




2

In [91]:
edge_dict = []
for i in range(40):
    farid = filters.farid(arr[i].data)
    # prewitt = filters.prewitt(arr[i])
    # roberts= filters.roberts(arr[i])
    # scharr = filters.scharr(arr[i])
    # sobel= filters.sobel(arr[i])
    otsu = filters.threshold_otsu(arr[i].data)
    thresh  = filters.threshold_otsu(arr[i].data)
    binary = (arr[i].data > thresh)*1
    negative = 1 - binary
    edge_data = {
        "farid":farid,
        # "prewitt":prewitt,
        # "roberts":roberts,
        # "scharr":scharr,
        # "c":sobel,
        "binary":binary,
        "negative":negative
    }
    edge_dict.append(edge_data)
    
df_edge = pd.DataFrame.from_dict(edge_dict)


In [111]:
data2 = []
for column in df_edge.columns:
    for i in range(40):
        edge = df_edge[column].iloc[i]
        data2.append(edge)

edges = np.array(data2)

In [112]:
all = np.array(data2)
all1 = all[0:120:2]
all2 = all[1:120:2]

In [80]:
dict_list1 = []

for i in range(int(60)):
    for use_mask in [True, False]:
        # for use_threshold in [True, False]:
        corr1, maxVal1, cen1, err1 = xcorr(all1[i], all2[i], use_mask, use_threshold=False)
        data_dict1 = {
        "n":i, 
        "use_mask":use_mask, 
        # "use_threshold":use_threshold,
        "corr1":corr1, 
        "maxVal1":maxVal1, 
        "cen1":cen1, 
        "err1_x":err1[0], 
        "err1_y":err1[1]}
        dict_list1.append(data_dict1)

df1 = pd.DataFrame.from_dict(dict_list1)

In [121]:
@dataclass
class Point:
    x: float = 0.0
    y: float = 0.0
def stretch(img, tilt_degrees):
    tilt = np.deg2rad(tilt_degrees)
    shape = int(img.shape[0] / np.cos(tilt)), int(img.shape[1] / np.cos(tilt))
    resized_img = np.asarray(Image.fromarray(img).resize(size=(shape[1], shape[0])))
    c = Point(resized_img.shape[1]//2, resized_img.shape[0]//2)
    dy, dx = img.shape[0]//2, img.shape[1]//2
    scaled_img = resized_img[c.y-dy:c.y+dy, c.x-dx:c.x+dx]
    return scaled_img

In [122]:
tilter = []
for i in range(60):
    # if not isinstance(all2[i], AdornedImage):
    #     img = AdornedImage(data = all2[i])
    # else:
    #     img = all2[i]
    cos = stretch(all2[i], 25)
    tilter.append(cos)

tilted = np.array(tilter)

In [127]:
dict_list2 = []

for i in range(int(60)):
    for use_mask in [True, False]:
        # for use_threshold in [True, False]:
        corr2, maxVal2, cen2, err2 = xcorr(all1[i], tilted[i], use_mask, use_threshold=False)
        data_dict2 = {
        "n":i, 
        "use_mask":use_mask, 
        # "use_threshold":use_threshold,
        "corr2":corr2, 
        "maxVal2":maxVal2, 
        "cen2":cen2, 
        "err2_x":err2[0], 
        "err2_y":err2[1]}
        dict_list2.append(data_dict2)

df2 = pd.DataFrame.from_dict(dict_list2)

In [138]:
display(df1)

,n,use_mask,use_threshold,corr1,maxVal1,cen1,err1_x,err1_y
0,0,True,True,"[[-16354.540464899614, -15102.246394878934, -1...",52726.858672,"[512.0, 768.0]",6,592
1,0,False,True,"[[10056.53937915373, 9917.735819654947, 9798.5...",42384.723059,"[512.0, 768.0]",-192,655
2,1,True,True,"[[4416.2340234193425, 3248.5174086442635, 2377...",51220.078804,"[512.0, 768.0]",-36,-667
3,1,False,True,"[[-815.9080429991113, -1357.3198617369253, -84...",42793.814827,"[512.0, 768.0]",-203,640
4,2,True,True,"[[-4770.061568801761, -5855.682515494284, -634...",61527.496068,"[512.0, 768.0]",-130,-53
...,...,...,...,...,...,...,...,...
115,57,False,True,"[[66800.38032214163, 68500.92526381004, 66336....",105604.815250,"[512.0, 768.0]",472,605
116,58,True,True,"[[-23051.701457868014, -23441.403577984885, -2...",134109.654475,"[512.0, 768.0]",-5,-7
117,58,False,True,"[[-883.1189237424005, -153.03932015460305, -21...",113835.653073,"[512.0, 768.0]",-34,-7
118,59,True,True,"[[-18472.873652688606, -16459.767056782857, -1...",67694.201219,"[512.0, 768.0]",-219,-560


In [139]:
display(df2)

,n,use_mask,corr2,maxVal2,cen2,err2_x,err2_y
0,0,True,"[[2712.019950748035, 2372.4759354926287, 2315....",47593.618081,"[512.0, 768.0]",-123,-255
1,0,False,"[[3431.6620221696026, 2997.116361175501, 2284....",41445.289796,"[512.0, 768.0]",-280,-468
2,1,True,"[[9895.586655372334, 8542.001826774092, 6983.8...",63751.784760,"[512.0, 768.0]",6,-70
3,1,False,"[[709.8806683703106, 1330.5163617964845, 2096....",49490.732590,"[512.0, 768.0]",-329,-478
4,2,True,"[[8853.110403513285, 7412.670108971205, 5728.8...",60936.711771,"[512.0, 768.0]",-85,-55
...,...,...,...,...,...,...,...
115,57,False,"[[68837.7000528968, 68878.46294738656, 69300.0...",113662.737017,"[512.0, 768.0]",469,622
116,58,True,"[[-1610.9000619847975, -1642.2766029621362, -1...",101091.972171,"[512.0, 768.0]",-66,-2
117,58,False,"[[-4630.748646546352, -5440.456055281695, -604...",166162.389382,"[512.0, 768.0]",-65,-5
118,59,True,"[[-7195.155615907428, -7375.31864409454, -7632...",71265.178062,"[512.0, 768.0]",41,-11


In [130]:

px.scatter(df1, "n", ["err1_x", "err1_y"], color="use_mask", trendline="ols")

In [131]:
px.scatter(df2, "n", ["err2_x", "err2_y"], color="use_mask", trendline="ols")

In [156]:
for i in range(40):
  notilt_errx = df1["err1_x"].iloc[i]
  notilt_erry = df1["err1_y"].iloc[i]  
for i in range(40):
  withtilt_errx = df2["err2_x"].iloc[i]
  withtilt_erry = df2["err2_y"].iloc[i]  


In [166]:
notilt_errx = []
notilt_erry = []
hastilt_errx = []
hastilt_erry = []
for i in range(40):
  notilt_errx.append(df1["err1_x"].iloc[i])
  notilt_erry.append(df1["err1_y"].iloc[i])
  hastilt_errx.append(df2["err2_x"].iloc[i])
  hastilt_erry.append(df2["err2_y"].iloc[i] ) 
  